# General Ideas
The task of this work is to locate the most likely locations where the excellent candidate may live, so that the company can send their recruiters to find her without wasting time or money. From the description of the question, we understood that the probability that the candidate live at a certain point is given by three unrelated probability distributions. Naturally, we are thinking to form a mixture probability distribution which could combine and represent the three individual distributions together. And then the locations where corresponded to the the maximum value in this mixture distruibution could give us an idea of where the candidate may live.

The methods to combine the probability from different distributions could be vairous and depends on conditons. From the book in [], one approach would be that the mixture distributions could be formulated by taking linear combinations of more basic distributions. Therefore, we consider the mixture probability desity function $P(X)$ of form as
$$P(X) = \sum_{i=1}^{N} {\pi_{i}\cdot p_i(X)} $$
where $N$ is the number of basic probability distributions, $p_i(X)$ is <em>ith</em> basic density function and the paramater $\pi_{i}$ is the corresponding coefficients. The parameter $\pi$ could also be understand as the <em>Priori probability </em> which indicates the probability that the random variable $X$ falls into the each basic distribution. For this reason, we can obtain that 
$$\sum_{i=1}^{N} {\pi_{i} =1}, 0 \leqslant p(X) \leqslant 1 $$

Back to this questions, we have three independent probability distributions. The first distribution is a normal distributions according to the shortest distance to river spree which could be denoted by $\mathcal{N}(\mu,\,\sigma^2)$. The second distribution is a log-normal distribution denoted by $\ln\mathcal{N}(\mu,\,\sigma^2)$ according to the shortest distance to the Brandenburg Gate. And the last one is also a normal distribution $\mathcal{N_3}(\mu,\,\sigma^2)$ according to the shortest distance to a satellite path. Since there is not enough information to calculate the parameter $\pi$ for each individual distribution, we made an assumption that the three distributions are equally important. So we get 
$$\pi_1 = \pi_2 = \pi_3 \quad and \quad \pi_1 + \pi_2 + \pi_3 = 1 \\ $$
In this case, we get the value of the coefficients $\pi_1 = \pi_2 = \pi_3 = \frac{1}{3}$. Finally, we could combine and represent the mixture probability density function for any given location (x,y) as following:
$$
P(x, y) = \pi_1\cdot f_1(d_1(x,y)))+\pi_2\cdot f_2(d_2(x,y)))+\pi_3\cdot f_3(d_3(x,y))) \\
=\frac{1}{3}\mathcal{N_1}(\mu,\,\sigma^2)+\frac{1}{3}\ln\mathcal{N_2}(\mu,\,\sigma^2)+\frac{1}{3}\mathcal{N_3}(\mu,\,\sigma^2)$$

The last step is to compute the maximum value for the mixture density function and the find the corresponding location. There will be many mathmatic considerations for calculating the maximum value from complex probability density function. We could invert the value of the complex probability function from positive space into the negative space and translate this problem to an optimization problem. The gradient descent algorithm is a widely used method to find a local minimum of a given objective function. By saying local minimum, we mean the algorithm may converge into a local minimum depending on the starting points. This local minimun makes no guarantee that it will be also the global minimum value. Luckily in this question, we can plot the probability density function of the interesting area and find all the local minimum values by carefully choose the starting point. After that, we can quickly pick the global minimum value from the local minimum list.


 # Coding
 We are trying to solve this problem using Python language.